In [1]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [278]:
data = pd.read_csv('data/botswana.tsv', sep = '\t')

In [279]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [280]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4361 entries, 0 to 4360
Data columns (total 15 columns):
ceb         4361 non-null int64
age         4361 non-null int64
educ        4361 non-null int64
religion    4361 non-null object
idlnchld    4241 non-null float64
knowmeth    4354 non-null float64
usemeth     4290 non-null float64
evermarr    4361 non-null int64
agefm       2079 non-null float64
heduc       1956 non-null float64
urban       4361 non-null int64
electric    4358 non-null float64
radio       4359 non-null float64
tv          4359 non-null float64
bicycle     4358 non-null float64
dtypes: float64(9), int64(5), object(1)
memory usage: 511.1+ KB


### 1. Сколько разных значений принимает признак religion?

In [360]:
data['religion'].unique()

array(['catholic', 'protestant', 'spirit', 'other'], dtype=object)

In [281]:
data['religion'].unique().shape

(4L,)

### 2.  Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски?

In [282]:
data.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1834 entries, 1 to 4360
Data columns (total 15 columns):
ceb         1834 non-null int64
age         1834 non-null int64
educ        1834 non-null int64
religion    1834 non-null object
idlnchld    1834 non-null float64
knowmeth    1834 non-null float64
usemeth     1834 non-null float64
evermarr    1834 non-null int64
agefm       1834 non-null float64
heduc       1834 non-null float64
urban       1834 non-null int64
electric    1834 non-null float64
radio       1834 non-null float64
tv          1834 non-null float64
bicycle     1834 non-null float64
dtypes: float64(9), int64(5), object(1)
memory usage: 229.2+ KB


### 3. В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному.
Используем для обработки пропусков в agefm и heduc следующий метод.

- Создайте признак nevermarr, равный единице там, где в agefm пропуски.
- Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, в нашей матрице X будет мультиколлинеарность.
- Замените NaN в признаке agefm на $c_{agefm}=0$.
- У объектов, где nevermarr = 1, замените NaN в признаке heduc на $c_{heduc_1}=−1$ (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).

**Сколько осталось пропущенных значений в признаке heduc?**

In [283]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [284]:
# Создайте признак nevermarr, равный единице там, где в agefm пропуски
data['nevermarr'] = data['agefm'].apply(lambda x: 1 if pd.isnull(x) else 0)

In [285]:
# Удалите признак evermarr
data.drop('evermarr', axis=1, inplace=True)

In [286]:
# Замените NaN в признаке agefm на  cagefm=0
data['agefm'].fillna(0, inplace=True)

In [287]:
# У объектов, где nevermarr = 1, замените NaN в признаке heduc на  cheduc1= −1
data.loc[(data['nevermarr'] == 1) & (data['heduc'].isnull()), 'heduc'] = -1

In [288]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [289]:
data['heduc'].isnull().sum()

123

### 4. Избавимся от оставшихся пропусков.

Для признаков idlnchld, heduc и usemeth проведите операцию, аналогичную предыдущей: создайте индикаторы пропусков по этим признакам (idlnchld_noans, heduc_noans, usemeth_noans), замените пропуски на нехарактерные значения ($c_{idlnchld}=−1$, $c_{heduc_2}=−2$ (значение -1 мы уже использовали), $c_{usemeth}=−1$).

Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. Их очень мало, так что удалите объекты, на которых их значения пропущены.

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).

In [291]:
# создаем индикаторы пропусков
data['idlnchld_noans'] = data['idlnchld'].apply(lambda x: 1 if pd.isnull(x) else 0)
data['heduc_noans'] = data['heduc'].apply(lambda x: 1 if pd.isnull(x) else 0)
data['usemeth_noans'] = data['usemeth'].apply(lambda x: 1 if pd.isnull(x) else 0)

In [292]:
# заменяем пропуски на нехарактерные значения
data['idlnchld'].fillna(-1, inplace=True)
data['heduc'].fillna(-2, inplace=True)
data['usemeth'].fillna(-1, inplace=True)

Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. Их очень мало, так что удалите объекты, на которых их значения пропущены.

In [293]:
data.dropna(subset=['knowmeth', 'electric', 'radio', 'tv', 'bicycle'], inplace=True)

In [294]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0,0,0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0,0,0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0,0,0,0


**Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).**

In [296]:
data.shape, data.size

((4348, 18), 78264)

### 5. 
Постройте регрессию количества детей ceb на все имеющиеся признаки методом smf.ols, как в разобранном до этого примере. Какой получился коэффициент детерминации $R^2$? Округлите до трёх знаков после десятичной точки.

In [346]:
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban +'
             'electric + radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data)
fitted1 = m1.fit()
print fitted1.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Thu, 26 Oct 2017   Prob (F-statistic):               0.00
Time:                        10:06:25   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Intercept                 -1

In [356]:
print 'критерий Бройша-Пагана:', sms.het_breushpagan(fitted1.resid, fitted1.model.exog)[1]

критерий Бройша-Пагана: 1.14529276334e-225


Вывод по критерию Бройша-Пагана на гомоскедастичность ошибки: нужна поправка Уайта.

### 8.
Удалите из модели незначимые признаки religion, radio и tv. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта.

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

In [348]:
m2 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban +'
             'electric + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data)
fitted2 = m2.fit(cov_type='HC1')
print fitted2.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Thu, 26 Oct 2017   Prob (F-statistic):               0.00
Time:                        10:06:52   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [355]:
print 'критерий Бройша-Пагана:', sms.het_breushpagan(fitted2.resid, fitted2.model.exog)[1]

критерий Бройша-Пагана: 1.11974588965e-228


Вывод по критерию Бройша-Пагана: поправка Уайта нужна.

In [338]:
print "Критерий Фишера: pv = %f" % m1.fit().compare_f_test(m2.fit())[1]

Критерий Фишера: pv = 0.467231


ПРИМЕЧАНИЕ по интерпретации критерия Фишера: проверяем нулевую гипотезу о том, что мы правильно убрали признаки (то есть качество модели не ухудшилось).

Вывод: поскольку $pv > 0.05$, то гипотеза о том, что мы правильно убрали признаки, не отвергается.

### 9.
Признак usemeth_noans значим по критерию Стьюдента, то есть, при его удалении модель значимо ухудшится. Но вообще-то отдельно его удалять нельзя: из-за того, что мы перекодировали пропуски в usemeth произвольно выбранным значением cusemeth=−1, удалять usemeth_noans и usemeth можно только вместе.

Удалите из текущей модели usemeth_noans и usemeth. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости (например, если вы получили $5.5×10^{−8}$, нужно ввести 8).

In [350]:
m3 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + agefm + heduc + urban +'
             'electric + bicycle + nevermarr + idlnchld_noans + heduc_noans', 
             data=data)
fitted3 = m3.fit(cov_type='HC1')
print fitted3.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     396.4
Date:                Thu, 26 Oct 2017   Prob (F-statistic):               0.00
Time:                        10:07:22   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         -1.1931      0.262     -4.

In [351]:
print 'критерий Бройша-Пагана:', sms.het_breushpagan(fitted3.resid, fitted3.model.exog)[1]

критерий Бройша-Пагана: 2.48563622975e-219


Вывод по критерию Бройша-Пагана: поправка Уайта нужна.

In [352]:
print 'Критерий Фишера: pv =', m2.fit().compare_f_test(m3.fit())[1]

Критерий Фишера: pv = 3.15520094804e-40


Вывод по Критерию Фишера: поскольку $pv < 0.05$, то гипотеза о том, что при убирании признаков качество модели не ухудшилось, отвергается. Поэтому возвращаем признаки обратно.

### 10.

In [359]:
m4 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban +'
             'electric + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data)
fitted4 = m4.fit(cov_type='HC1')
print fitted4.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Thu, 26 Oct 2017   Prob (F-statistic):               0.00
Time:                        10:27:37   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [358]:
print 'критерий Бройша-Пагана:', sms.het_breushpagan(fitted4.resid, fitted4.model.exog)[1]

критерий Бройша-Пагана: 1.11974588965e-228


Вывод по критерию Бройша-Пагана: нужна поправка Уайта.